<a href="https://colab.research.google.com/github/FanmeiWang/Dynamic-Fulfillment-ChatBot/blob/main/Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

app = Flask(__name__)

# Load sensitive data from environment variables
STUDENT_NUMBER = os.getenv("STUDENT_NUMBER", "not-set")  # Default to 'not-set' if missing
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN", None)

# Route to return student number in JSON format
@app.route('/')
def student_number():
    return jsonify({"student_number": STUDENT_NUMBER})

# Dialogflow Webhook route
@app.route('/webhook', methods=['POST'])
def webhook():
    # Get the request payload
    req = request.get_json(force=True)
    intent_name = req.get('queryResult').get('intent').get('displayName')  # Get the intent name

    # If the intent is GetStudentNumber
    if intent_name == 'GetStudentNumber':
        return jsonify({
            'fulfillmentText': f'Your student number is {STUDENT_NUMBER}'
        })

    # If the intent is AskHobbies
    elif intent_name == 'AskHobbies':
        # Get the full text input from the user
        user_input = req.get('queryResult').get('queryText')
        hobby_keywords = {
            "sports": ["sports", "tennis", "basketball", "football", "climbing", "hiking"],
            "music": ["music", "guitar", "piano", "singing", "band", "violin"],
            "reading": ["reading", "books", "novels", "fiction", "biographies"]
        }
        # Default response
        response_text = "I'm not sure I understood your hobby. Can you tell me more about it?"

        # Match keywords to categories and generate responses dynamically
        for category, keywords in hobby_keywords.items():
            if any(keyword in user_input.lower() for keyword in keywords):  # Check if any keyword matches
                if category == "sports":
                    response_text = "You seem to love sports! Have you tried exploring hiking trails or playing tennis with friends?"
                    break
                elif category == "music":
                    response_text = "You enjoy music! Perhaps you could join a local band or take piano lessons?"
                    break
                elif category == "reading":
                    response_text = "Books are amazing! Have you checked out the latest sci-fi novels or classic biographies?"
                    break

        return jsonify({
            'fulfillmentText': response_text
        })

    # If the intent is HobbyFollowUp
    elif intent_name == 'HobbyFollowUp':
        # Handle follow-up questions
        user_reply = req.get('queryResult', {}).get('queryText', '').lower()

        if user_reply == "yes":
            response_text = "Great! Here are some more ideas for you to explore. Have fun!"
        elif user_reply == "no":
            response_text = "No problem! Thank you for chatting with me. Have a great day!"
        else:
            response_text = "Please reply with 'yes' or 'no'."

        return jsonify({
            'fulfillmentText': response_text
        })

    # Default fallback response
    else:
        return jsonify({
            'fulfillmentText': "Sorry, I don't understand."
        })

# Start ngrok to expose Flask app
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

# Start Flask app
app.run(port=5000)
